In [52]:
import numpy as np
import pickle
import os
import sys
import pandas as pd
import plotly.express as px
sys.path.insert(1, os.path.abspath('src/env'))
from hparams import HParams
from envfactory import WindturbineEnv

In [167]:
with open('src/env/reward_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [170]:
np.linalg.norm([0,0])

0.0

In [125]:
def get_column_names(hparams):
    orig_state_columns = ["rotational speed [rad/s]", 
    "power [W]", 
    "HH wind velocity [m/s]", 
    "yaw angle [deg]", 
    "pitch blade 1 [deg]", 
    "pitch blade 2 [deg]", 
    "pitch blade 3 [deg]", 
    "oop blade root bending moment blade 1 [N]", 
    "oop blade root bending moment blade 2 [N]", 
    "oop blade root bending moment blade 3 [N]", 
    "ip blade root bending moment blade 1 [N]", 
    "ip blade root bending moment blade 2 [N]", 
    "ip blade root bending moment blade 3 [N]", 
    "tor blade root bending moment blade 1 [Nm]", 
    "tor blade root bending moment blade 2 [Nm]", 
    "tor blade root bending moment blade 3 [Nm]", 
    "oop tip deflection blade 1 [m]", 
    "oop tip deflection blade 2 [m]", 
    "oop tip deflection blade 3 [m]", 
    "ip tip deflection blade 1 [m]", 
    "ip tip deflection blade 2 [m]", 
    "ip tip deflection blade 3 [m]", 
    "tower top acceleration in global X [m^2/s]",
    "tower top acceleration in global Y [m^2/s]",
    "tower top acceleration in global Z [m^2/s]",
    "tower bottom bending moment along global X [Nm]",
    "tower bottom bending moment along global Y [Nm]",
    "tower bottom bending moment along global Z [Nm]",
    "current time [s]",
    "azimuthal position of the LSS [deg]",
    "azimuthal position of the HSS [deg]",
    "HSS torque [Nm]",
    "wind speed at hub height [m/s]",
    "horizontal inflow angle [deg]",
    "tower top fore aft acceleration [m/s^2]",
    "tower top side side acceleration [m/s^2]",
    "tower top X position [m]",
    "tower top Y position [m]]",
    "controller torque [N]",
    "controller pitch 1 [deg]",
    "controller pitch 2 [deg]",
    "controller pitch 3 [deg]"]
    
    state_columns = orig_state_columns
    if hparams and "coleman_transform" in hparams and hparams["coleman_transform"]:
        state_columns = list(np.append(state_columns, ["coleman transformed bending moments d", "coleman transformed bending moments q"]))
    if hparams and "polar_transform" in hparams and "polar_transform_lss" in hparams and hparams["polar_transform_lss"]:
        state_columns = list(np.append(state_columns, ["polar transformed LSS position [x]", "polar transformed LSS position [y]"]))
    if hparams and "polar_transform" in hparams and "polar_transform_hss" in hparams and hparams["polar_transform_hss"]:
        state_columns = list(np.append(state_columns, ["polar transformed LSS position [x]", "polar transformed LSS position [y]"]))

    if hparams and "mask" in hparams and hparams["mask"] and "mask_obs" in hparams:
        state_columns = list(np.array(state_columns)[hparams["mask_obs"]])
    if hparams and "dilated_past_feeding" in hparams and hparams["dilated_past_feeding"]:
        new_columns = []
        for i in range(hparams["dilated_past_feeding_steps"]):
            tmp_columns = [x + " p%d" % i for x in state_columns]
            new_columns.append(tmp_columns)
        state_columns = list(itertools.chain(*new_columns))

    orig_action_columns = ["torque", "pitch_1", "pitch_2", "pitch_3"]
    action_columns = orig_action_columns
    if hparams and "mask" in hparams and hparams["mask"] and "mask_act" in hparams:
        if hparams["mask_act"] == "individual-pitch-no-torque":
            action_columns = ["pitch_1", "pitch_2", "pitch_3"]
        elif hparams["mask_act"] == "collective_pitch":
            action_columns = ["torque", "pitch"]
    
    return state_columns, action_columns, orig_state_columns, orig_action_columns

In [152]:
hparams = HParams(
  randomize_wind=False,
  clip_action_gradients=True,
  differential_control=True,
  coleman_transform=False,
  log_env=False,
  normalizing=False,
  mean_normalizing=False,
  mask=False,
  averaging=False,
  log_masked=False,
  freeze_actions=False,
  past_feeding=False,
  dilated_past_feeding=False,
  enable_ignore_steps=False,
  enable_trajectory_clip=False,
  delayed_rewards=False
)
global_env = WindturbineEnv(hparams)

In [153]:
state_columns, _, _, _ = get_column_names(hparams)

In [154]:
first_ipc_rollout = pd.DataFrame(np.array(data_ideal['states'])[0], columns=state_columns)

In [146]:
np.array(data_ideal['states'])[0].shape

(2000, 44)

In [164]:
px.line(0.09 - np.linalg.norm(dq, axis=1)*3e-7)

In [133]:
px.line(first_rollout, y=["oop blade root bending moment blade 1 [N]", 
    "oop blade root bending moment blade 2 [N]", 
    "oop blade root bending moment blade 3 [N]"])

In [134]:
px.line(first_rollout, y=['coleman transformed bending moments d', 'coleman transformed bending moments q'])

In [135]:
px.line(first_rollout, y=["pitch blade 1 [deg]", 
    "pitch blade 2 [deg]", 
    "pitch blade 3 [deg]"])

In [78]:
  def coleman_transf(azi,MY1,MY2,MY3,order=1):
    col_mat = 2.0/3*np.array([[np.cos(order*azi), np.cos(order*(azi+2*np.pi/3)), np.cos(order*(azi+4*np.pi/3))],
                              [np.sin(order*azi), np.sin(order*(azi+2*np.pi/3)), np.sin(order*(azi+4*np.pi/3))]])
    return np.dot(col_mat,np.array([MY1,MY2,MY3]))
  def inv_coleman_transf(azi,lead,DQ,max_pit=None,min_pit=None,order=1):
    inv_col_mat = np.array([[np.cos(order*(azi+lead)), np.sin(order*(azi+lead))],
                            [np.cos(order*(azi+lead+2*np.pi/3)), np.sin(order*(azi+2*np.pi/3))],
                            [np.cos(order*(azi+lead+4*np.pi/3)), np.sin(order*(azi+lead+4*np.pi/3))]])
    if max_pit and min_pit:
      DQ = np.clip(DQ,min_pit/np.sqrt(2),max_pit/np.sqrt(2))
    return np.dot(inv_col_mat,DQ)


In [155]:
azi = np.array(first_ipc_rollout["azimuthal position of the LSS [deg]"])
azi = np.deg2rad(azi)
MY1 = np.array(first_ipc_rollout["oop blade root bending moment blade 1 [N]"])
MY2 = np.array(first_ipc_rollout["oop blade root bending moment blade 2 [N]"])
MY3 = np.array(first_ipc_rollout["oop blade root bending moment blade 3 [N]"])

dq = np.array([coleman_transf(azi[i], MY1[i], MY2[i], MY3[i]) for i in range(200)])

In [156]:
px.line(dq)

In [117]:
reverse = np.array([inv_coleman_transf(azi[i], 1, (np.sin(azi[i]),np.sin(azi[i]))) for i in range(200)])
px.line(reverse)

In [113]:
reverse = np.array([inv_coleman_transf(azi[i], 0, (np.random.normal(size=(2), loc=1, scale=0.1))) for i in range(200)])
px.line(reverse)

In [92]:
inv_coleman_transf(2, 0, (0,1))

array([ 0.90929743, -0.81504245, -0.09425498])

In [101]:
inv_coleman_transf(2, np.deg2rad(-90), (1, 0))

array([ 0.90929743, -0.81504245, -0.09425498])

In [102]:
np.deg2rad(90)

1.5707963267948966